# <center> TP2 - 01 Description

# Objective of TP

In this TP you will develop the full **supervised learning pipeline** including the *hyper-parameter tuning* and *model evalutaion*.  

You will then apply the pipeline to three algorithms
* nearest neighbour
* decision tree
* default classifier

Finally, you will perform *model comparison* and **discuss** its results.

### Recommendation:
The code you will develop in this TP is to be re-used in TP3 and the exam.  
Therefore we recommend you try to make it clear (use comments, when printing say what you print) so that next time it is easier for you to remember what it does.  
Also, try to make the code generic so that it can be easilly used for different datasets.   
Try to automate as much as possible so that the code does not require too much of your attention, finally you will need to do the same type of analysis not for 3 algorithms but for 5-6.

## Dataset

You will be workig with the same cars dataset as in TP1.  
Each group shall be using the same `brands` as in TP1.


In [1]:
# Load dataset and extract our part
import pandas as pd

# Reading csv file
autos = pd.read_csv('autos.csv',encoding='latin-1')

# Extracting the relevant part for our group
only_specific_brands = autos.brand.isin(['renault', 'peugeot', 'skoda', 'citroen', 'ford'])
autos = autos[only_specific_brands]

In [48]:
autos.head()

,price,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,fuel_type,brand,unrepaired_damage,fast_sale
2,11400.0,limousine,2010,manuell,175.0,mondeo,125000.0,diesel,ford,False,False
4,4100.0,kleinwagen,2009,manuell,68.0,1_reihe,90000.0,benzin,peugeot,False,False
6,888.0,kombi,2000,manuell,115.0,mondeo,150000.0,benzin,ford,False,True
7,13700.0,bus,2012,manuell,86.0,roomster,5000.0,benzin,skoda,False,True
9,4299.0,kleinwagen,2010,manuell,75.0,2_reihe,125000.0,benzin,peugeot,False,False


# Data preprocessing

Remember that after loading the dataset, there are several preprocessing steps you need to do before trainign the algorithm.
If you are not sure what these are, see *Course 8 - 02 Hyper-parameter tuning*.

When writing the code, **put short comments explaining what the pre-processing steps are and why you need to do them**.

### First we are changing the column names:

- We changed the style of writing the column names from camelcase to snakecase. This change will make column names easier to read, and we won't have to remember which letter is capitalized.
- We reworded some of the column names in order for them to be more descriptive and obvious.

In [3]:
# Changing the column names
autos.columns = ['price', 'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model', 
                 'kilometer', 'fuel_type', 'brand', 'unrepaired_damage', 'fast_sale']
autos.head()

,price,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,fuel_type,brand,unrepaired_damage,fast_sale
2,11400.0,limousine,2010.0,manuell,175.0,mondeo,125000.0,diesel,ford,nein,False
4,4100.0,kleinwagen,2009.0,manuell,68.0,1_reihe,90000.0,benzin,peugeot,nein,False
6,888.0,kombi,2000.0,manuell,115.0,mondeo,150000.0,benzin,ford,nein,True
7,13700.0,bus,2012.0,manuell,86.0,roomster,5000.0,benzin,skoda,nein,True
9,4299.0,kleinwagen,2010.0,manuell,75.0,2_reihe,125000.0,benzin,peugeot,nein,False


### Next, we are going to convert some columns data type:

- We are changing data type in order to facilitate the data processing (classifying the data in numeric or categorical)
- We are converting column 'unrepaired_damage' from object to boolean type
- We are converting column 'registration_year' from float to int.

In [4]:
# Converting column 'unrepaired_damage' from object to boolean type
autos['unrepaired_damage'] = (autos['unrepaired_damage']
                 .str.replace('nein','')
                 .str.replace('ja','True')
                  .astype(bool)
                 )

In [5]:
# Converting column 'registration_year' from float to int.
autos['registration_year'] = (autos['registration_year'].astype(int))

# Is it relevant or we can simply keep it in float format ?

In [6]:
autos.info()
autos.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28401 entries, 2 to 166073
Data columns (total 11 columns):
price                28401 non-null float64
vehicle_type         28401 non-null object
registration_year    28401 non-null int64
gearbox              28401 non-null object
power_ps             28401 non-null float64
model                28401 non-null object
kilometer            28401 non-null float64
fuel_type            28401 non-null object
brand                28401 non-null object
unrepaired_damage    28401 non-null bool
fast_sale            28401 non-null bool
dtypes: bool(2), float64(3), int64(1), object(5)
memory usage: 2.2+ MB


,price,registration_year,power_ps,kilometer
count,28401.000000,28401.000000,28401.000000,28401.000000
mean,4177.515017,2003.965565,102.402979,121472.307313
std,4680.629533,5.872460,40.664873,39816.529262
min,1.000000,1923.000000,2.000000,5000.000000
25%,1199.000000,2001.000000,75.000000,100000.000000
50%,2500.000000,2004.000000,101.000000,150000.000000
75%,5400.000000,2008.000000,122.000000,150000.000000
max,73500.000000,2016.000000,952.000000,150000.000000


### Next, we are goint to decide what is output (target) and what is input:
- We do this because ...
- How we do it ...

In [7]:
# output dataframe
out_autos = autos['fast_sale']
# input dataframe
in_autos = autos.iloc[:,[0,1,2,3,4,5,6,7,8,9]]

print('Inputs')
display(in_autos.head())
print('Outputs')
display(out_autos.head())

Inputs


,price,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,fuel_type,brand,unrepaired_damage
2,11400.0,limousine,2010,manuell,175.0,mondeo,125000.0,diesel,ford,False
4,4100.0,kleinwagen,2009,manuell,68.0,1_reihe,90000.0,benzin,peugeot,False
6,888.0,kombi,2000,manuell,115.0,mondeo,150000.0,benzin,ford,False
7,13700.0,bus,2012,manuell,86.0,roomster,5000.0,benzin,skoda,False
9,4299.0,kleinwagen,2010,manuell,75.0,2_reihe,125000.0,benzin,peugeot,False


Outputs


2    False
4    False
6     True
7     True
9    False
Name: fast_sale, dtype: bool

### Next, we convert categorical to numeric (one-hot encoding)

- The algorithms we use (except the deciscion tree) cannot operate on label data directly. They require all input variables and output variables to be numeric. Therefore we need to convert categorical data to a numerical form.
- Since no ordinal relationship exists in the categorical variables in our dataset (and we don't want to allow the model to assume a natural ordering between categories), we use the one-hot encoding. 

In [8]:
print('Original input data')
print('-------------------')
display(in_autos.head())

# Create one-hot encoding
in_long = pd.get_dummies(in_autos)
print('\n Numerical input data')
print('-------------------')
display(in_long.head())

Original input data
-------------------


,price,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,fuel_type,brand,unrepaired_damage
2,11400.0,limousine,2010,manuell,175.0,mondeo,125000.0,diesel,ford,False
4,4100.0,kleinwagen,2009,manuell,68.0,1_reihe,90000.0,benzin,peugeot,False
6,888.0,kombi,2000,manuell,115.0,mondeo,150000.0,benzin,ford,False
7,13700.0,bus,2012,manuell,86.0,roomster,5000.0,benzin,skoda,False
9,4299.0,kleinwagen,2010,manuell,75.0,2_reihe,125000.0,benzin,peugeot,False



 Numerical input data
-------------------


,price,registration_year,power_ps,kilometer,unrepaired_damage,vehicle_type_andere,vehicle_type_bus,vehicle_type_cabrio,vehicle_type_coupe,vehicle_type_kleinwagen,...,fuel_type_cng,fuel_type_diesel,fuel_type_elektro,fuel_type_hybrid,fuel_type_lpg,brand_citroen,brand_ford,brand_peugeot,brand_renault,brand_skoda
2,11400.0,2010,175.0,125000.0,False,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,4100.0,2009,68.0,90000.0,False,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
6,888.0,2000,115.0,150000.0,False,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,13700.0,2012,86.0,5000.0,False,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,4299.0,2010,75.0,125000.0,False,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


### Next, we normalize (bring all variables to same scale [0,1])

-  We need to normalize the data to ensure that all sources are treated equally, and that data-availability bias (and its corresponding misrepresentation of the data universe) is reduced.

In [9]:
# maximum of each attribute
maxs = in_long.max(axis=0) # max accross rows (axis=0)

print('Maximums')
print('----------')
print(maxs)

# minimum of each attribute
mins = in_long.min(axis=0) # max accross rows (axis=0)

print('\n Minimums')
print('----------')
print(mins)

# min-max normalization
norm_in = (in_long - mins) / (maxs-mins)

# new maximum of each attribute
maxs = norm_in.max(axis=0) # max accross rows (axis=0)

print('\n New maximums')
print('----------')
print(maxs)

# new minimum of each attribute
mins = norm_in.min(axis=0) # max accross rows (axis=0)

print('\n New minimums')
print('----------')
print(mins)

Maximums
----------
price                       73500.0
registration_year            2016.0
power_ps                      952.0
kilometer                  150000.0
unrepaired_damage               1.0
vehicle_type_andere             1.0
vehicle_type_bus                1.0
vehicle_type_cabrio             1.0
vehicle_type_coupe              1.0
vehicle_type_kleinwagen         1.0
vehicle_type_kombi              1.0
vehicle_type_limousine          1.0
vehicle_type_suv                1.0
gearbox_automatik               1.0
gearbox_manuell                 1.0
model_1_reihe                   1.0
model_2_reihe                   1.0
model_3_reihe                   1.0
model_4_reihe                   1.0
model_5_reihe                   1.0
model_andere                    1.0
model_b_max                     1.0
model_berlingo                  1.0
model_c1                        1.0
model_c2                        1.0
model_c3                        1.0
model_c4                        1.0
model_c5

### As of this step, no more use of original data and only use preprocessed

In our case above: `out_autos` and `norm_in`

# Prepare for model evaluation and hyper-parameter tuning

### Data splits for model evaluation (training and testing)

You will need to write the code splitting the data to training set (used for model learning and hyper-parameter tuning) and test set used for final model evaluation (test error).

Here, you can choose to **use either 5-folds cross-validation or 5 time repeated hold-out method.**

**Tell us what your choice is and why**. Both choices are good, we just want to know that you understand the differences and you have thought about them.

Remember that in the end this procedure will be used for all your algorithms and that these should work over the same train/test splits. You can make sure this will be the case by fixing the seed for the random sample generation.

### 5-folds cross-validation
- We chose cross-validation because it generally results in a less biased or less optimistic estimate of the model skill than other methods, such as a simple train/test split (hold-out).
- The more folds we have, the more we will be reducing the error due the bias but increasing the error due to variance; the computational price would go up too, obviously — the more folds you have, the longer it would take to compute it and you would need more memory. With a lower number of folds, we’re reducing the error due to variance, but the error due to bias would be bigger. It’s would also computationally cheaper

### Data splits for hyper-parameter tuning

Here we want you to **use 3-folds inner cross validation**.

You will need to write the code to split each of the training sets above to train/validation accoridng to the 3-fold cross-validation strategy.

### Generalization accuracy

You will also need to prepere the code that will use the trained models to produce predictions for the test instances, calculate the accuracy of over each test set, and calculate the final average accuracy over all the test instances (estimate of generalization accuracy).

In [50]:
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn import neighbors, tree
import operator


class HyperParameterTuner:
    
    RANDOM_STATE = 123
    
    def __init__(self, in_data, out_data, hyper_params, 
                 clf_type, n_split_outer=5, n_split_inner=3):
        self._in_data = in_data
        self._out_data = out_data
        self._hyper_params = hyper_params
        self.n_split_outer = n_split_outer
        self.n_split_inner = n_split_inner
        
        # Binding the callback for the classifier received.
        if isinstance(clf_type, neighbors.KNeighborsClassifier):
            self._clf_callback = self.clf_neighbors_callback
            print("knn")
        elif isinstance(clf_type, tree.DecisionTreeClassifier):
            self._clf_callback = self.clf_tree_callback
            print("tree") 
        
    
    def clf_neighbors_callback(self, hyper_param):
        return neighbors.KNeighborsClassifier(n_neighbors=hyper_param)


    def clf_tree_callback(self, hyper_param):
        return tree.DecisionTreeClassifier(max_leaf_nodes=hyper_param)
    
        
    def dict_filled_with_zeros_for_each(self, keys):
        _dict = dict()
        for key in keys:
            _dict[key] = 0.0
        return _dict
            
        
    def run(self):
        self._pre_run_init()
        self.cross_validation()
        self.calculate_avg_of_hyper_params()
        
    
    def _pre_run_init(self):
        # Dictionnary holding the cross-validation (cv) scores.
        self._cv_scores = self.dict_filled_with_zeros_for_each(self._hyper_params)
        self._best_hyper_param_by_fold = dict()
        
    
    def cross_validation(self):
        k_fold = KFold(n_splits=self.n_split_outer, random_state=self.RANDOM_STATE, shuffle=True)
        for train_idx, test_idx in k_fold.split(self._in_data, self._out_data):
            in_train = self._in_data.iloc[train_idx]
            out_train = self._out_data.iloc[train_idx]
            out_test = self._out_data.iloc[test_idx]
            self._inner_cross_validation(in_train, out_train)
            self._print_best_hyper_param_by_fold()

    
    def _inner_cross_validation(self, in_train, out_train):
        for hp in self._hyper_params:
            clf = self._clf_callback(hp)
            scores = cross_val_score(clf, in_train, out_train, cv=self.n_split_inner)
            self._cv_scores[hp] += scores.mean() 
            self._best_hyper_param_by_fold[hp] = scores.mean() 

    
    def _print_best_hyper_param_by_fold(self):
        best_hp = max(self._best_hyper_param_by_fold.items(), key=operator.itemgetter(1))[0]
        print ("The optimal hyper-parameter is %d" % best_hp)
                
    
    def calculate_avg_of_hyper_params(self):
        for hyper_param, accuracy in self._cv_scores.items():
            self._cv_scores[hyper_param] = accuracy / self.n_split_outer

    
    def get_best_hyper_param(self):
        # Get the hyper-param with the highest accuracy in scores dict.
        return max(self._cv_scores.items(), key=operator.itemgetter(1))[0]
        
    
    def get_hyper_params_avg_scores(self):
        return self._cv_scores

In [15]:
class DefaultClassifier:
    
    def __init__(self):
        self.majority_class = None
        
    def fit(self, in_train, out_train):
        # get counts per class
        class_counts = out_train.value_counts()
        self.majority_class = class_counts.idxmax()
        
    def predict(self, out_test):
        # predict
        pred_test = out_test.copy()
        pred_test[:] = self.majority_class
        return pred_test

In [16]:
class FT:
    
    def __init__(self, in_data, out_data, hyper_param, clf_callback, n_splits=5):
        self.in_data=in_data
        self.out_data=out_data
        self.hyper_param=hyper_param
        self.clf_callback=clf_callback
        self.n_splits=n_splits
        self.out_test=list()
        self.pred_test=list()
    
    def run(self):
        # Use the learned model f to do predictions
        k_fold = KFold(n_splits=self.n_splits, random_state=123, shuffle=True)

        avg_test_accuracy = 0

        for train_idx, test_idx in k_fold.split(self.in_data, self.out_data):
            in_train = self.in_data.iloc[train_idx]
            out_train = self.out_data.iloc[train_idx]
            in_test = self.in_data.iloc[test_idx]
            self.out_test = self.out_data.iloc[test_idx]

            clf = self.clf_callback(self.hyper_param)
            clf.fit(in_train, out_train)

            # Prediction for all test data
            if isinstance(clf, DefaultClassifier):
                print("DefaultClassifier")
                self.pred_test = clf.predict(self.out_test)
            else:
                self.pred_test = clf.predict(in_test)

            # Get test set accuracy        
            test_accuracy = calculate_pred_accuracy(self.pred_test, self.out_test)
            print('Test accuracy', test_accuracy)
            avg_test_accuracy += test_accuracy

        # Use test sets to get estimate of generalization error (accuracy)
        print(avg_test_accuracy/self.n_splits)
        return (avg_test_accuracy/self.n_splits)

    
    def clf_neighbors(self, hyper_param):
        return neighbors.KNeighborsClassifier(n_neighbors=hyper_param)


    def clf_tree(self, hyper_param):
        return tree.DecisionTreeClassifier(max_leaf_nodes=hyper_param)

    
    def get_test_data(self):
        return (self.out_test, self.pred_test)
    
    
    def calculate_pred_accuracy(self, pred_data, out_data):
        match_test = (pred_data == out_data)
        accuracy = match_test.sum() / match_test.count()
        return accuracy

# Train and test nearest neigbour model

Once you have the general procedure in place, train the nearest neigbour model.

### Hyper-parameter search

Hyper-parameter in nearest neighbour algorithm is the number of neighbours to use.
We want you to try at least 5 different values. **Tell us which values you decide to try.** (There is no Why questoin here.)

Remember that for choosing the best hyper-parameter value, you use the the inner cross validation and the best hyper-parameter is the one with the highest average accuracy over the validation sets.

### Model lerning and test accuracy

Once you have the best value of the hyper-parameter, you use it to **retrain** the model over the merged train+validation (you do this 5 times, see above *Data splits for model evaluation*). You then use this **retrained** model to get the final test accuracy.

For each of the test samples (there should be 5, see above), report the test accuracy and the corresponding hyper-parameter setting (the one chosen as best for this specific split).

Are the hyper-parameter parameters the same for all the test sets? **Discuss** if you think this is  normal or not, why it happens and if it creates some difficulties for interpreting the model. **There is no single correct answer here!** We want to see that you undertand the procedure and that you use your brain.



In [11]:
knn_hpt = HyperParameterTuner(norm_in, out_autos, hyper_params=[2, 5, 15, 29, 50, 100], clf_type=neighbors.KNeighborsClassifier())
knn_hpt.run()
print(knn_hpt.get_hyper_params_avg_scores())
best_hp_knn = knn_hpt.get_best_hyper_param()
print(best_hp_knn)

knn
The optimal hyper-parameter is 100
The optimal hyper-parameter is 29
The optimal hyper-parameter is 29
The optimal hyper-parameter is 50
The optimal hyper-parameter is 50
{2: 0.5266805912214303, 5: 0.5381590445551269, 15: 0.5392414946683048, 29: 0.5442236399449584, 50: 0.5430089244341343, 100: 0.5425513321420768}
29


### Comment
- We found that the best hyper-parameter value is 13 , because it has the highest average accuracy over the validation sets.

In [34]:
def clf_neighbors(hyper_param):
    return neighbors.KNeighborsClassifier(n_neighbors=hyper_param)


knn_ft = FT(
    norm_in, 
    out_autos, 
#     hyper_param=best_hp_knn, 
    hyper_param=5, 
    clf_callback=clf_neighbors
)

knn_ft.run()
knn_pred_test, knn_out_test = knn_ft.get_test_data()

Test accuracy 0.5416299947192396
Test accuracy 0.5556338028169014
Test accuracy 0.5360915492957746
Test accuracy 0.553169014084507
Test accuracy 0.5426056338028169
0.5458259989438479


# Train and test decision tree

Use the same general procedure to train a decision tree.

Hyper-parameters for decision trees are the pre-prunning criteria such as maximum number of leafs (see *Course 5 - 02 Decision tree prunning*). 
Pick one of these and use at least 5 different values. **Tell us which one you pick and what values you are using.**

Calculate and report the test accuracies together with their corresponding heper-parameter values. (No more comments needed here.)

In [51]:
dtree_hpt = HyperParameterTuner(norm_in, out_autos, hyper_params=[2, 5, 15, 29, 50, 100], clf_type=tree.DecisionTreeClassifier())
dtree_hpt.run()
print(dtree_hpt.get_hyper_params_avg_scores())
best_hp_tree = dtree_hpt.get_best_hyper_param()
print(best_hp_tree)

tree
(5681,)
The optimal hyper-parameter is 2
(5680,)
The optimal hyper-parameter is 29
(5680,)
The optimal hyper-parameter is 29
(5680,)
The optimal hyper-parameter is 15
(5680,)
The optimal hyper-parameter is 5
{2: 0.5764851210364509, 5: 0.5775942155387808, 15: 0.5776734699442199, 29: 0.5781047467541509, 50: 0.5760361471473007, 100: 0.5738618126214481}
29


In [35]:
def clf_tree(hyper_param):
    return tree.DecisionTreeClassifier(max_leaf_nodes=hyper_param)


tree_ft = FT(
    norm_in, 
    out_autos, 
#     hyper_param=best_hp_knn, 
    hyper_param=5,
    clf_callback=clf_tree
)

tree_ft.run()
tree_pred_test, tree_out_test = tree_ft.get_test_data()

Test accuracy 0.5675057208237986
Test accuracy 0.5757042253521126
Test accuracy 0.578169014084507
Test accuracy 0.5808098591549296
Test accuracy 0.5619718309859155
0.5728321300802526


# Train and test default classifier

Default classifier has no hyper-parameters, so you can skip the inner-cross validatoin procedure.

Calculate and report the test accuracies for the 5 test sets from the part *Data splits for model evaluation*

In [36]:
def clf_default(hyper_param): #need to pass zero
    return DefaultClassifier()


dc_ft = FT(
    norm_in, 
    out_autos, 
    hyper_param=0,
    clf_callback=clf_default
)

dc_ft.run()
dc_pred_test, dc_out_test = dc_ft.get_test_data()

DefaultClassifier
Test accuracy 0.5118817109663791
DefaultClassifier
Test accuracy 0.5121478873239437
DefaultClassifier
Test accuracy 0.5184859154929577
DefaultClassifier
Test accuracy 0.5126760563380282
DefaultClassifier
Test accuracy 0.5235915492957747
0.5157566238834167


# Compare models

Once you have all your test accuracies for the nearest neighbour, decision trees and default classifier, calculate the estimated generalization accuracy of each (the everage accuracy accross the test sets).

Is any of the algorithms peforming better than the other two? **Discuss, comment.**

## Use the McNemar test 

Use the McNemar test to verify whether the differences in the generalization accuracy are significant. 

In McNemar you can always compare only two algorithms. Do all the pair-wise comparisons, present and **explain** the results. Are these what you would expect?

In [70]:
from statsmodels.stats.contingency_tables import mcnemar
import matplotlib.pyplot as plt 

def mc_nemar_test(m1, m2, alpha=0.05):
    ctab = pd.crosstab(m1,m2)
    #display(ctab)
    # calculate mcnemar test
    result = mcnemar(ctab)
    # interpret test results
    
    display(pd.DataFrame(ctab))

    # Crosstab for fast_sale frequencies
    
    if result.pvalue < alpha:
        print('p-value < alpha: at {} significance level we reject H0. We conclude that M1 and M2 differ.'.format(alpha))
    else:
        print('p-value >= alpha: at {} significance level we cannot reject H0. We cannot say that M1 and M2 differ.'.format(alpha))

In [71]:
# Default classifier vs KNN

m1 = np.logical_and(dc_pred_test, dc_out_test)  
m2 = np.logical_and(knn_pred_test, knn_out_test) 

mc_nemar_test(m1, m2)

fast_sale,False,True
fast_sale,,
False,4006,0
True,0,1674


p-value >= alpha: at 0.05 significance level we cannot reject H0. We cannot say that M1 and M2 differ.


In [72]:
# Default Classifier vs Descision Tree

m1 = np.logical_and(dc_pred_test, dc_out_test)  
m2 = np.logical_and(tree_pred_test, tree_out_test)

mc_nemar_test(m1, m2)

fast_sale,False,True
fast_sale,,
False,3724,0
True,0,1956


p-value >= alpha: at 0.05 significance level we cannot reject H0. We cannot say that M1 and M2 differ.


In [73]:
# KNN vs Descision Tree

m1 = np.logical_and(tree_pred_test, tree_out_test)
m2 = np.logical_and(knn_pred_test, knn_out_test) 

mc_nemar_test(m1, m2)

fast_sale,False,True
fast_sale,,
False,4006,0
True,0,1674


p-value >= alpha: at 0.05 significance level we cannot reject H0. We cannot say that M1 and M2 differ.


# Comparing the models

### Write an observation for each algorithm and compare performance
#### ==>
### Explain WHY some algorithms are better than the others
#### ==>

# The McNemar test

### Nearest neighbour vs Decision trees 
#### Result
#### Explaination
### Nearest neighbour vs Default classifier
#### Result
#### Explaination
### Decision trees vs Default classifier
#### Result
#### Explaination

KNN is unsupervised, Decision Tree (DT) supervised. (KNN is supervised learning while K-means is unsupervised, I think this answer causes some confusion.) KNN is used for clustering, DT for classification.(Both are used for classification.)

KNN determines neighborhoods, so there must be a distance metric. This implies that all features must be numeric. Distance metrics may be effected by varying scales between attributes and also high-dimensional space.

DT on the other hand predicts a class for a given input vector. The attributes may be numeric or nominal.

So, if you want to find similar examples you could use KNN. If you want to classify examples you could use DT.


In this paper, five common classifier families were tested to discriminate six classes of lung tissue patterns in HRCT data from healthy cases and cases affected with ILDs. Evaluation of the classifiers is based on a high-quality dataset taken from clinical routine. The classifiers were optimized in order to compare their best performance. The SVM classifier constitutes the best trade-off between the error rate on the training set and generalization, the ability to classify ROIs correctly from images of new patients. Since SVMs were designed to avoid overfitting of training samples, using them to classify medical images with much heterogeneity is adapted. The SVM classifier was able to correctly classify 88.3% of the instances into the six classes and 96.4% when discriminating healthy tissue versus all other pathological classes. Two metrics were used to characterize the performances of the classifiers: scores based on McNemar’s test along with global accuracy on the test set. The two metrics have shown to be complementary. The optimal classification algorithms were integrated into a software for classification of ROIs directly in three-dimensional DICOM images (Fig. 9). The diagnostic aid tool is easy to integrate into the PACS having the same user interface and offers the possibility to add clinical data from the electronic patient record. The classifier belongs to the core of a computer-aided diagnosis system involved in the decision-making process.

Larger values of K will have smoother decision boundaries which means lower variance but increased bias.